In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests
from lxml import html
import json
import numpy as np

# Get Players

In [3]:
dump = requests.get('https://fantasy.premierleague.com/drf/bootstrap')
load = dump.text

In [4]:
json_dump = json.loads(load)

In [5]:
datapoints = ['id', 'web_name', 'first_name', 'total_points', 'event_points', 'now_cost','cost_change_start', 
              'selected_by_percent', 'form','ep_next', 
              'chance_of_playing_next_round','minutes', 'bonus', 'goals_scored', 'assists', 'clean_sheets', 
              'goals_conceded', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'ea_index', 'news']

player_list = []

for player in range(0, len(json_dump['elements'])):
    stats = [json_dump['elements'][player][x] for x in datapoints]
    player_list.append(stats)
    
player_df = pd.DataFrame(player_list)

# Rename columns

player_df.columns = datapoints

# Get Positions

In [6]:
players = requests.get('https://fantasy.premierleague.com/player-list/')
totals = players.text
total_soup = BeautifulSoup(totals)

In [7]:
#Get all player names

player_names = total_soup.find_all('td')[::4]

players = []

for player in player_names:
    name = player.get_text()
    players.append(name)

player_name_df = pd.DataFrame({"Name": players})

In [8]:
# Positions for first and last player in each field

for i, e in enumerate(player_name_df['Name']):
    if e == 'De Gea':   
        gk=i
    if e == 'Azpilicueta':   
        defe=i
    if e == 'Salah':   
        mid=i
    if e == 'Kane':   
        att=i

In [9]:
#Populate df with positions
player_name_df['position']=''


player_name_df.loc[gk:(defe),'position']='Goalkeeper'
player_name_df.loc[defe:(mid),'position']='Defender'
player_name_df.loc[mid:(att),'position']='Midfielder'
player_name_df.loc[att:,'position']='Attacker'

In [10]:
fpl_players =  pd.merge(player_name_df, player_df, left_on='Name', right_on='web_name')

In [11]:
# Deal with duplicates

duplicats = fpl_players['Name'].value_counts()>1

dup_names = fpl_players['Name'].isin(['McCarthy','Ward','Bennett','Davies','Gray','Long','Simpson','Reid','Sánchez','McCartyh','Stephens', 'Pereira', 'Murphy', 'Williams'])
dup_ids = fpl_players['id'].isin([295, 359])


fpl_players.drop(fpl_players[dup_names].index, inplace=True)
fpl_players.drop(fpl_players[dup_ids].index, inplace=True)

/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/algorithms.py:410: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  f = lambda x, y: htable.ismember_object(x, values)
/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


# Optimal Choice

In [13]:
import pulp
import regex

In [760]:
#Make variables

#Player variables by position
influence = fpl_players[['id',"influence"]]
threat = fpl_players[['id',"threat"]]
creativity = fpl_players[['id',"creativity"]]

players = range(0, len(fpl_players))


gks = fpl_players[fpl_players['position']=='Goalkeeper']
defs = fpl_players[fpl_players['position']=='Defender']
mds = fpl_players[fpl_players['position']=='Midfielder']
ats = fpl_players[fpl_players['position']=='Attacker']

rgks=range(0, len(gks))
rdefs=range(0, len(defs))
rmds=range(0, len(mds))
rats=range(0, len(ats))

In [25]:
# Formula to evaluate

def total_influence(player):
    """Find the happiness of the table - 
    by calculating the combined influence between the players"""
    return sum(fpl_players[fpl_players['id']==player[0]]['influence'].astype(float),fpl_players[fpl_players['id']==player[-1]]['influence'].astype(float))

In [761]:
#Set constraint of variables
max_gks = 2
max_money = 1000

#create list of all possible tables https://www.ce.teiep.gr/e-class/modules/document/file.php/216/PAPERS/2009.%20Pulp%20For%20Python%20Programmers.pdf
possible_gks = [tuple(c) for c in pulp.allcombinations(gks['id'], max_gks)]

x = pulp.LpVariable('x', upBound=max_money, cat='Continuous')

# Look for all players used
y = pulp.LpVariable.dicts('keeper_used', players, lowBound=0,upBound=1, cat=pulp.LpInteger)

#Indicator variable, showing if player been used
g = pulp.LpVariable.dicts('keeper_used', test, lowBound=0,upBound=1, cat=pulp.LpInteger)
#create a binary variable to state that a player combo is used
gu = pulp.LpVariable.dicts('gks', possible_gks, lowBound = 0, upBound = 1,cat = pulp.LpInteger)
#d = pulp.LpVariable.dicts('d', rdefs, 0,1, pulp.LpInteger)
#m = pulp.LpVariable.dicts('m', rmds, 0,1, pulp.LpInteger)
#a = pulp.LpVariable.dicts('a', rats, 0,1, pulp.LpInteger)

In [762]:
#Create optimisation problem

player_choice = pulp.LpProblem("Player choice model", pulp.LpMaximize)

player_choice += pulp.lpSum([ y[player]*
                             #influence.loc[influence['id']==player]['influence'].astype(float) 
                             float(influence.iloc[player]['influence']) 
                             for player in players])

In [29]:
#Create Constraints

#Only 2 players
player_choice += sum([y[player] for player in players]) <= 2

In [24]:
seating_model += sum([x[table] for table in possible_tables]) <= max_tables,

1

In [35]:
#Solving 
player_choice.solve()

for v in player_choice.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
    else:
        continue

('keeper_used_211', '=', 1.0)
('keeper_used_392', '=', 1.0)


# Iteration 2.0

In [789]:
#Change format of df

fpl_players['influence_f'] = fpl_players['influence'].astype(float)

#Make variables

#Player variables by position
influence = fpl_players[['id',"influence_f"]]
threat = fpl_players[['id',"threat"]]
creativity = fpl_players[['id',"creativity"]]


gks = fpl_players[fpl_players['position']=='Goalkeeper']
defs = fpl_players[fpl_players['position']=='Defender']
mds = fpl_players[fpl_players['position']=='Midfielder']
ats = fpl_players[fpl_players['position']=='Attacker']

rgks=range(0, len(gks))
rdefs=range(0, len(defs))
rmds=range(0, len(mds))
rats=range(0, len(ats))

In [791]:
# Formula to evaluate

def total_influence(player):
    """Find the happiness of the table - 
    by calculating the combined influence between the players"""
    return sum(fpl_players[fpl_players['id']==player[0]]['influence'].astype(float),fpl_players[fpl_players['id']==player[-1]]['influence'].astype(float))

In [854]:
#Set constraint of variables
max_money = 1000

#create list of all possible tables https://www.ce.teiep.gr/e-class/modules/document/file.php/216/PAPERS/2009.%20Pulp%20For%20Python%20Programmers.pdf

#x = pulp.LpVariable('x', upBound=max_money, cat='Continuous')

# Look for all gks used
gv = pulp.LpVariable.dicts('goalkeeper_used', rgks, lowBound=0,upBound=1, cat=pulp.LpInteger)
dv = pulp.LpVariable.dicts('defender_used', defs['id'], lowBound=0,upBound=1, cat=pulp.LpInteger)
mv = pulp.LpVariable.dicts('midfielder_used', mds['id'], lowBound=0,upBound=1, cat=pulp.LpInteger)
av = pulp.LpVariable.dicts('attacker_used', ats['id'], lowBound=0,upBound=1, cat=pulp.LpInteger)

In [856]:
#Create optimisation problem

player_choice = pulp.LpProblem("Player choice model", pulp.LpMaximize)

player_choice += pulp.lpSum([ gv[gm]*
                             influence.loc[influence['id']==gm]['influence_f']
                             for gm in gks['id']]
                           +[ dv[dm]*
                             influence.loc[influence['id']==dm]['influence_f']
                             for dm in defs['id']] 
                           +[ mv[mm]*
                             influence.loc[influence['id']==mm]['influence_f'] 
                             for mm in mds['id']]
                           +[ av[am]*
                             influence.loc[influence['id']==am]['influence_f']
                             for am in ats['id']]
                           )

KeyError: 282

In [857]:
# Constraints

#Exactly 2 keepers
player_choice += sum([gv[player] for player in gks['id']]) == 2

#Exactly 4 defenders
player_choice += sum([dv[player] for player in defs['id']]) == 4

#Exactly 4 midfielders
player_choice += sum([mv[player] for player in mds['id']]) == 4

#Exactly 3 attackers
player_choice += sum([av[player] for player in ats['id']]) == 3

KeyError: 282

# Iteration 3.0

In [399]:
#Change format of df
fpl_players['influence_f'] = fpl_players['influence'].astype(float)
fpl_players['influence_f'].fillna(0, inplace=True)

fpl_players['bps_f'] = fpl_players['bps'].astype(float)
fpl_players['bps_f'].fillna(0, inplace=True)

#Make variables

#Player variables by position
influence = fpl_players[['id',"influence_f"]]
threat = fpl_players[['id',"threat"]]
creativity = fpl_players[['id',"creativity"]]


gks = fpl_players[fpl_players['position']=='Goalkeeper']
defs = fpl_players[fpl_players['position']=='Defender']
mds = fpl_players[fpl_players['position']=='Midfielder']
ats = fpl_players[fpl_players['position']=='Attacker']

rgks=range(0, len(gks))
rdefs=range(0, len(defs))
rmds=range(0, len(mds))
rats=range(0, len(ats))

# Possible tuples
possible_gk = [tuple(c) for c in pulp.allcombinations(gks['id'], 2)]

In [400]:
#Make optjection function

def total_influence(player, number_players):
    """Find the value of the table - 
    by calculating the combined influence between the players"""
    if len(player)==number_players:
        return sum(fpl_players[fpl_players['id']==player[0]]['bps'],fpl_players[fpl_players['id']==player[-1]]['bps']).item()
    else:
        return 0


In [401]:
# Make variables

# for all possible GK options
z = pulp.LpVariable.dicts('goalkeepers', possible_gk, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

In [402]:
#Optimization goal

player_selection = pulp.LpProblem("Player selection model", pulp.LpMaximize)

#player_selection += [total_influence(keepers) * x[keepers] for keepers in possible_gk]
player_selection += pulp.lpSum([total_influence(keepers,2) * z[keepers] for keepers in possible_gk]), 'Max influence'

In [403]:
# Get right number of players
player_selection += pulp.lpSum([z[keepers] for keepers in possible_gk]) == 1, "Maximum_number_of_tables"

#player_selection += ([len(keepers) for keepers in possible_gk]) == 2, "Number of keepers"

In [404]:
player_selection.solve()

1

In [405]:
player_selection.objective.values()

[757,
 757,
 1423,
 1451,
 1384,
 757,
 805,
 1458,
 811,
 1481,
 1349,
 1086,
 1139,
 799,
 783,
 972,
 774,
 1388,
 1483,
 757,
 1098,
 757,
 757,
 1263,
 757,
 757,
 757,
 1458,
 793,
 1037,
 929,
 1108,
 757,
 1151,
 757,
 1392,
 1131,
 1529,
 757,
 757,
 757,
 757,
 757,
 932,
 757,
 811,
 757,
 666,
 694,
 627,
 48,
 701,
 54,
 724,
 592,
 329,
 382,
 42,
 26,
 215,
 17,
 631,
 726,
 341,
 506,
 701,
 36,
 280,
 172,
 351,
 394,
 635,
 374,
 772,
 175,
 54,
 666,
 694,
 627,
 48,
 701,
 54,
 724,
 592,
 329,
 382,
 42,
 26,
 215,
 17,
 631,
 726,
 341,
 506,
 701,
 36,
 280,
 172,
 351,
 394,
 635,
 374,
 772,
 175,
 54,
 1360,
 1293,
 666,
 714,
 1367,
 720,
 1390,
 1258,
 995,
 1048,
 708,
 692,
 881,
 683,
 1297,
 1392,
 666,
 1007,
 666,
 666,
 1172,
 666,
 666,
 666,
 1367,
 702,
 946,
 838,
 1017,
 666,
 1060,
 666,
 1301,
 1040,
 1438,
 666,
 666,
 666,
 666,
 666,
 841,
 666,
 720,
 666,
 1321,
 694,
 742,
 1395,
 748,
 1418,
 1286,
 1023,
 1076,
 736,
 720,
 909,
 711,
 

In [406]:
pulp.LpStatus[player_selection.status]

'Optimal'

In [407]:
for team in possible_gk:
    if z[team].value() == 1.0:
        print(team)
        print fpl_players[fpl_players['id']==team[0]]['Name']
        print fpl_players[fpl_players['id']==team[-1]]['Name']  

(282, 400)
0    De Gea
Name: Name, dtype: object
58    Fabianski
Name: Name, dtype: object


In [409]:
fpl_players[fpl_players['position']=='Goalkeeper'].sort_values(by='bps_f', ascending=False)

,Name,position,id,web_name,first_name,total_points,event_points,now_cost,cost_change_start,selected_by_percent,...,goals_conceded,bps,influence,creativity,threat,ict_index,ea_index,news,influence_f,bps_f
58,Fabianski,Goalkeeper,400,Fabianski,Lukasz,157,0,45,0,20.8,...,56,772,1087.8,10.0,0.0,109.9,0,,1087.8,772.0
0,De Gea,Goalkeeper,282,De Gea,David,172,0,60,0,29.5,...,28,757,821.2,10.0,0.0,83.4,0,,821.2,757.0
19,Ryan,Goalkeeper,47,Ryan,Mathew,146,0,45,0,6.1,...,54,726,911.6,0.0,0.0,91.1,0,,911.6,726.0
10,Pickford,Goalkeeper,154,Pickford,Jordan,145,0,50,0,21.7,...,58,724,873.6,30.1,4.0,90.7,0,,873.6,724.0
28,Lössl,Goalkeeper,191,Lössl,Jonas,135,0,45,0,2.4,...,58,701,846.8,30.0,0.0,87.8,0,,846.8,701.0
8,Pope,Goalkeeper,68,Pope,Nick,152,0,50,0,0.8,...,35,701,862.4,40.0,0.0,90.1,0,Shoulder Dislocation - Unknown return date,862.4,701.0
4,Lloris,Goalkeeper,351,Lloris,Hugo,144,0,55,0,8.4,...,35,694,726.4,0.1,0.0,72.3,0,,726.4,694.0
3,Ederson,Goalkeeper,260,Ederson,Ederson,158,0,55,0,17.7,...,26,666,546.2,10.0,0.0,55.6,0,,546.2,666.0
56,Foster,Goalkeeper,445,Foster,Ben,123,0,45,0,4.5,...,55,635,716.8,20.0,0.0,73.8,0,,716.8,635.0
18,Begovic,Goalkeeper,24,Begovic,Asmir,112,0,45,0,4.9,...,61,631,790.8,20.0,0.0,80.8,0,,790.8,631.0


# Get output

In [1101]:
player_choice.solve()

1

In [1129]:
for v in player_choice.variables():
    if v.varValue > 0:
        print(fpl_players[fpl_players['id']==int(v.name.split('_')[-1])]['position'],":",
            fpl_players[fpl_players['id']==int(v.name.split('_')[-1])]['Name'], "=", v.varValue)
    else:
        continue


In [172]:
items = [("a", 5),
         ("b", 6),
         ("c", 7),
         ("d", 32),
         ("e", 2),
         ("f", 32),
         ("g", 5),
         ("h", 7),
         ("i", 9),
         ("k", 12),
         ("l", 11),
         ("m", 1),
         ("n", 2)]

itemCount = len(items)

# Max number of bins allowed.
maxBins = 32

# Bin Size
binCapacity = 32



# Indicator variable assigned 1 when the bin is used.
y = pulp.LpVariable.dicts('BinUsed', range(maxBins),
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

# An indicator variable that is assigned 1 when item is placed into binNum
possible_ItemInBin = [(itemTuple, binNum) for itemTuple in items
                                            for binNum in range(maxBins)]

In [ ]:
possible_ItemInBin = [(itemTuple, binNum) for itemTuple in items
                                            for binNum in range(maxBins)]

In [173]:
possible_ItemInBin

[(('a', 5), 0),
 (('a', 5), 1),
 (('a', 5), 2),
 (('a', 5), 3),
 (('a', 5), 4),
 (('a', 5), 5),
 (('a', 5), 6),
 (('a', 5), 7),
 (('a', 5), 8),
 (('a', 5), 9),
 (('a', 5), 10),
 (('a', 5), 11),
 (('a', 5), 12),
 (('a', 5), 13),
 (('a', 5), 14),
 (('a', 5), 15),
 (('a', 5), 16),
 (('a', 5), 17),
 (('a', 5), 18),
 (('a', 5), 19),
 (('a', 5), 20),
 (('a', 5), 21),
 (('a', 5), 22),
 (('a', 5), 23),
 (('a', 5), 24),
 (('a', 5), 25),
 (('a', 5), 26),
 (('a', 5), 27),
 (('a', 5), 28),
 (('a', 5), 29),
 (('a', 5), 30),
 (('a', 5), 31),
 (('b', 6), 0),
 (('b', 6), 1),
 (('b', 6), 2),
 (('b', 6), 3),
 (('b', 6), 4),
 (('b', 6), 5),
 (('b', 6), 6),
 (('b', 6), 7),
 (('b', 6), 8),
 (('b', 6), 9),
 (('b', 6), 10),
 (('b', 6), 11),
 (('b', 6), 12),
 (('b', 6), 13),
 (('b', 6), 14),
 (('b', 6), 15),
 (('b', 6), 16),
 (('b', 6), 17),
 (('b', 6), 18),
 (('b', 6), 19),
 (('b', 6), 20),
 (('b', 6), 21),
 (('b', 6), 22),
 (('b', 6), 23),
 (('b', 6), 24),
 (('b', 6), 25),
 (('b', 6), 26),
 (('b', 6), 27),
